# TP2: Estimación de peso y dimensiones de los envíos de Mercado Libre

# Materia: Análisis y curación de datos

## Análisis del dataset

Se pide: A partir de lo visto en la teoría de la materia y del segundo laboratorio, diagramar una comunicación en formato textual o interactivo describiendo la solución de las actividades propuestas a continuación. Al final de las mismas se proveen actividades opcionales (no obligatorias) que pueden resultar de interés.

### Actividades Propuestas:
    1.	Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.

    2.	Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.

    3.	Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.

    4.	Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.

    5.	Transformar variables categóricas en números (Se recomienda OneHotEncoding) para las columnas (Sugerencia: arrancar con un sample de ~10K items)

        a.	`CATALOG_PRODUCT_ID`
        b.	`CONDITION`
        c.	`DOMAIN_ID`
        d.	`SELLER_ID`
        e.	`BRAND` (extraída en 4)
        f.	`MODEL`(extraída en 4)

    6.	En caso de tener alguna variable no medida (en nuestro caso `PRICE`) imputar sus valores utilizando kNN.

    7.	Medir las distribuciones de las variables como histogramas, realizar normalizaciones e identificar outliers con los métodos vistos en clase. Hacer análisis de estos outliers y considerar si sería correcto o no eliminarlos del dataset. Sugerencia: Identificar outliers de las columnas `SHP_WEIGHT` y `SHP_VOLUME`, donde `SHP_VOLUME` se define como el producto de las dimensiones.

    8.	[Opcional] Si están interesados en usar texto y técnicas de NLP sobre el título:
        a.	Realizar preprocesamiento del texto, se recomienda leer este artículo.
        b.	Para convertir el título en vectores se recomienda ver tf-idf. wiki, sklearn.

    9.	[Opcional] Aplicar PCA y decidir si existe un subespacio del espacio de features donde viven los datos y son bien descriptos. Graficar.

    10.	[Opcional] Aplicar Mixturas Gaussianas (GMM) para hacer clustering. 

La comunicación debe estar apuntada a un público técnico pero sin conocimiento del tema particular, como por ejemplo, sus compañeros de clase o stakeholders del proyecto. Idealmente, además del documento se debería generar una presentación corta para stakeholders explicando el análisis realizado sobre los datos y las conclusiones obtenidas de tal análisis.

Se evaluarán los siguientes aspectos:

    ●	El informe debe contener un mensaje claro y presentado de forma concisa.
    ●	Los gráficos deben aplicar los conceptos de percepción visual vistos en clase.
    ●	Se debe describir o estimar la significancia estadística de su trabajo.


In [ ]:
# pip install scikit-learn

In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import scipy as sc
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from ast import literal_eval
from pandas.io.json import json_normalize
#import json

In [40]:
random.seed(0)

El dataset corresponde a un muestreo de envíos durante tres meses de 2018 en MLB (Mercado Libre Brasil), filtrados según métodos de envíos y por carriers (empresas de correos a cargo del envío propiamente dicho). A su vez, fue muestreado de forma no uniforme para romper de forma manual la distribución de los datos.
Los datos son:[meli_dataset_20190426.csv] (https://drive.google.com/open?id=1tNUKD1lf1z8C7LPpCruiDl6SKBAHb79v).

In [4]:
DATASET = 'meli_dataset_20190426.csv'

In [33]:
df_original = pd.read_csv(DATASET, low_memory=False)


In [6]:
df= df_original

In [7]:
df = df.head(10000) 

In [8]:
df.columns

Index(['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'ATTRIBUTES', 'CATALOG_PRODUCT_ID', 'CONDITION', 'DOMAIN_ID', 'PRICE',
       'SELLER_ID', 'STATUS', 'TITLE'],
      dtype='object')

#### ¿Qué significa cada columna?

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.

In [9]:
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
8463,MXDZ4UPRCY,1000.0,40.0,30.0,15.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",OU13D2QR2Y,new,MLB-SURVEILLANCE_CAMERAS,79.99,DLGC80MT2Q,active,Câmera Hikvision Bullet Hd Hd-tvi 720p 3.6mm D...
9462,Z19DXRNYPQ,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-LEGGINGS,29.00,PW528QRH8H,active,Legging Suplex Grossa Flanelada Cós Alto Estam...
8510,B8JSYIFR1C,NaN,NaN,NaN,NaN,NaN,H53U1H7Q5G,NaN,NaN,NaN,OISK1WO7IW,under_review,Frete Grátis Point H - Maquininha De Cartão Do...
7356,G6DUP0UOHS,518.0,25.0,15.0,11.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-ELECTRONIC_PRODUCTS,62.50,NIRDPY3H98,active,Mini Bomba De Água + Fonte 12v 2a + 2mts Mangu...
3976,ZRMQ7A53S7,281.0,25.0,25.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-T_SHIRTS,169.00,T4RS0GWSSP,active,Camiseta Importada / T-shirt / Burberry / Pra...


In [10]:
df.dtypes

ITEM_ID                object
SHP_WEIGHT            float64
SHP_LENGTH            float64
SHP_WIDTH             float64
SHP_HEIGHT            float64
ATTRIBUTES             object
CATALOG_PRODUCT_ID     object
CONDITION              object
DOMAIN_ID              object
PRICE                 float64
SELLER_ID              object
STATUS                 object
TITLE                  object
dtype: object

## Actividad 1:

Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.


In [11]:
# df.filter(["STATUS"])!= "404"

df = df[df.STATUS != "404"]
df = df.drop(columns=['STATUS'])
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,TITLE
1711,BDGCOC4E3Q,380.0,50.1,5.3,4.7,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-DECORATIVE_VINYLS,18.40,OVYSK5MAFY,Papel Adesivo Contact Colorido 100cm X 50cm Cores
2372,OUGXMI31NC,2030.0,45.0,30.0,25.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-CAR_SEAT_COVERS,140.00,EBRFYMSBGG,Capa Couro Ecológico F250 F350 F1000 F4000 Vár...
1756,PEEPEWRKM1,81.0,16.0,11.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",HAWTSRVH85,new,MLB-CELLPHONE_LCD_SCREENS,33.22,FJ6V3JACW0,Tela Touch Galaxy Gran Prime Duos Sm-g531bt Sm...
3301,D0JIPEZ7IQ,520.0,16.0,11.0,4.0,"[{'id': 'BICYCLE_PRODUCT_TYPE', 'name': 'Tipo ...",H53U1H7Q5G,new,MLB-BICYCLE_TOOL_KITS,39.50,EQAM0NBMQN,Kit Ferramentas Bike Chaves Remendo Reparo Pne...
8303,LE76IDPIS4,39.0,0.0,0.0,0.0,"[{'id': 'BRAND', 'name': 'Marca da Capa', 'val...",H53U1H7Q5G,new,MLB-CELLPHONE_COVERS,16.90,EW40K57RFR,Capinha Celular Nomes Personalizada Samsung J5...


## Actividad 2:

Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.


In [12]:
# df = df.dropna(subset=['SHP_WEIGHT'])
# df = df.dropna(subset=['SHP_LENGTH'])
# df = df.dropna(subset=['SHP_WIDTH'])
# df = df.dropna(subset=['SHP_HEIGHT'])

# o bien:
df = df.dropna(subset=['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT'])

#o bien

# df= df.dropna(df.filter(regex ='SHP_'))


#df.sample(3)



## Actividad 3:

Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.

In [13]:
medianas = df.groupby('ITEM_ID').median()
medianas


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE
ITEM_ID,,,,,
A0BKGTRNTM,775.0,50.0,20.0,10.0,750.00
A0JIHFAMOP,6100.0,70.0,25.0,5.0,119.90
A0NG260OMQ,464.0,20.0,11.0,10.0,349.90
A0WL6O455O,150.0,25.0,25.0,11.0,21.99
A1957258HH,3719.0,42.0,34.0,13.0,849.00
...,...,...,...,...,...
ZZFFUBJ43D,3880.0,36.0,24.0,13.0,145.90
ZZGF7K64VC,1609.0,22.5,20.0,18.0,185.00
ZZQD93N8HV,200.0,23.0,20.5,6.5,NaN


## Actividad 4:

Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.

In [101]:
def attribute_to_list(attribute):
    if attribute:
        return literal_eval(attribute)
    else:
        return []

In [102]:
# myString = df["ATTRIBUTES"][1].strip('[ ]')
# attribute_to_list(myString)
# json_normalize(attribute_to_list(myString))

In [103]:
#attribute_to_list(df["ATTRIBUTES"][1])

In [104]:
json_normalize(attribute_to_list(df["ATTRIBUTES"][1]))

,id,name,value_id,value_name,value_struct,attribute_group_id,attribute_group_name
0,BRAND,Marca,None,Manipulados,None,OTHERS,Outros
1,GTIN,Código universal de produto,-1,None,None,OTHERS,Outros
2,ITEM_CONDITION,Condição do item,2230284,Novo,None,OTHERS,Outros
3,LINE,Linha,436920,HomeCare,None,OTHERS,Outros
4,MODEL,Modelo,None,Manipulados,None,OTHERS,Outros
5,SKIN_CARE_SUPPLY_FUNCTIONS,Funções do artigo,None,Clareador,None,OTHERS,Outros


In [88]:
#datos = myString.replace("'", '"')

# FALTA EXTRAER BRAND Y MODEL EN CADA CASO Y LUEGO ITERAR!!!!!!!!!!!!!!!!


## Actividad 5:

Transformar variables categóricas en números (Se recomienda OneHotEncoding) para las columnas (Sugerencia: arrancar con un sample de ~10K items)

In [145]:
df.sample(2)
# luego agregar las variables generadas a partir de atributes

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,TITLE
3204,LU540OAB00,40.0,16.0,11.0,4.0,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-DC_POWER_JACK_CONNECTORS,3599.0,C28968C1L8,Dc Jack Dell Inspiron 14 15-3458 3558 5455 545...
2761,PG3T10S15T,74.0,32.0,23.0,12.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-FLEA_AND_TICK_TREATMENTS,119.0,DQ74OKRU3S,"Bravecto Antipulgas E Carrapatos Cães Até 4,5 ..."


In [146]:
# Exploración de cantidad de categorías distintas
print(pd.DataFrame(df.CATALOG_PRODUCT_ID.unique()).count())
print(pd.DataFrame(df.CONDITION.unique()).count())
print(pd.DataFrame(df.DOMAIN_ID.unique()).count())
print(pd.DataFrame(df.SELLER_ID.unique()).count())
print(pd.DataFrame(df.TITLE.unique()).count())

0    485
dtype: int64
0    3
dtype: int64
0    908
dtype: int64
0    3380
dtype: int64
0    4377
dtype: int64


In [148]:
column = 'CATALOG_PRODUCT_ID'
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column])
#pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
CATALOG_PRODUCT_ID_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))
CATALOG_PRODUCT_ID_ENCODED.sample(5)

,CATALOG_PRODUCT_ID_A0J8EHPW8Y,CATALOG_PRODUCT_ID_A0RY70BE19,CATALOG_PRODUCT_ID_A2H2JJFBXM,CATALOG_PRODUCT_ID_A4M0AP2TSK,CATALOG_PRODUCT_ID_A6X73QCLS9,CATALOG_PRODUCT_ID_A7UXO0015Y,CATALOG_PRODUCT_ID_A7Y7QKJ7EF,CATALOG_PRODUCT_ID_ADKMKF0FVM,CATALOG_PRODUCT_ID_AF4WQUGCVH,CATALOG_PRODUCT_ID_AFPLIBE9VN,...,CATALOG_PRODUCT_ID_ZD9PH6CQGT,CATALOG_PRODUCT_ID_ZE1SAQN048,CATALOG_PRODUCT_ID_ZHPQEDV8OO,CATALOG_PRODUCT_ID_ZK6EHO4XIX,CATALOG_PRODUCT_ID_ZNCNVOCZCI,CATALOG_PRODUCT_ID_ZOLFL65EFH,CATALOG_PRODUCT_ID_ZSQJ90IBB0,CATALOG_PRODUCT_ID_ZUPYDUNVT5,CATALOG_PRODUCT_ID_ZUZF6Q39W8,CATALOG_PRODUCT_ID_ZZDRE28WA4
2266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4873,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
df["CONDITION"]

1       new
2       new
3       new
5       new
7       new
       ... 
9990    new
9995    NaN
9996    NaN
9997    NaN
9998    new
Name: CONDITION, Length: 5088, dtype: object

In [153]:
column = "CONDITION"
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column].astype(str))
pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
CONDITION_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))
CONDITION_ENCODED.sample(5)

,CONDITION_nan,CONDITION_new,CONDITION_not_specified,CONDITION_used
1030,0,1,0,0
2807,0,1,0,0
1291,0,1,0,0
3377,0,1,0,0
2144,0,1,0,0


In [157]:
column = 'DOMAIN_ID'
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column].astype(str))
#pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
DOMAIN_ID_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))
DOMAIN_ID_ENCODED.sample(5)

,DOMAIN_ID_MLB-ABS_SENSORS,DOMAIN_ID_MLB-ACOUSTIC_GUITARS,DOMAIN_ID_MLB-ACOUSTIC_PANELS,DOMAIN_ID_MLB-ACTION_FIGURES,DOMAIN_ID_MLB-ADHESIVE_TAPES,DOMAIN_ID_MLB-AEROBICS_AND_FITNESS_EQUIPMENT,DOMAIN_ID_MLB-AIRGUN_PELLETS,DOMAIN_ID_MLB-AIRSOFT_GUNS,DOMAIN_ID_MLB-AIR_CONDITIONER_REMOTE_CONTROLS,DOMAIN_ID_MLB-AIR_FRESHENERS,...,DOMAIN_ID_MLB-WIRELESS_ANTENNAS,DOMAIN_ID_MLB-WIRELESS_CHARGERS,DOMAIN_ID_MLB-WIRELESS_FM_TRANSMITTERS,DOMAIN_ID_MLB-WIRE_STRIPPERS,DOMAIN_ID_MLB-WOMEN_SWIMWEAR,DOMAIN_ID_MLB-WRENCHES,DOMAIN_ID_MLB-WRENCH_SETS,DOMAIN_ID_MLB-WRISTWATCHES,DOMAIN_ID_MLB-XENON_KITS,DOMAIN_ID_nan
1796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
746,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
column = 'SELLER_ID'
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column].astype(str))
#pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
SELLER_ID_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))
SELLER_ID_ENCODED.sample(5)

,SELLER_ID_A0C377D6IK,SELLER_ID_A0G4VA0XM9,SELLER_ID_A0GTKIHE1A,SELLER_ID_A0KW5KL0S5,SELLER_ID_A115K1RP2T,SELLER_ID_A1QE03ELPN,SELLER_ID_A2K70P3I18,SELLER_ID_A2PJ265JWZ,SELLER_ID_A3I3O11AE1,SELLER_ID_A4RK3WZHUP,...,SELLER_ID_ZU5MPG54HI,SELLER_ID_ZUM6UCCICB,SELLER_ID_ZVW99W8TBU,SELLER_ID_ZW0R4QP48J,SELLER_ID_ZWDY6BPHGS,SELLER_ID_ZXCKFJHKGT,SELLER_ID_ZXFTE5ZZ00,SELLER_ID_ZY07CQVW6O,SELLER_ID_ZYUC8GTM87,SELLER_ID_ZZCYH6OP0M
171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2182,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2838,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
column = 'TITLE'
lb = LabelBinarizer()
lb_results = lb.fit_transform(df[column].astype(str))
#pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_)).head(10)
TITLE_ENCODED = pd.DataFrame(lb_results, columns=(column + '_') + pd.Series(lb.classes_))
TITLE_ENCODED.sample(5)

,TITLE_ Unifi Ponto De Acesso Ao Ar Livre 802.11ac,TITLE_ 10 Regata Masculina Dry Fit Poliester Esportes Academia,TITLE_ 3 Composto Natural Dieta Multi Ervas 60 Caps,TITLE_ Armação Para Óculos De Grau Retrô Vintage Quadrado Geek,TITLE_ Blusa Listra Tecido Moletom Ref.15349,TITLE_ Bota Coturno Feminina Atacado 7 Pares Revenda,TITLE_ Bota Feminina Montaria Cano Alto Full Jln741,TITLE_ Bucha Do Eixo E Do Amortecedor Traseiro Do Peugeot 206 Heat,TITLE_ Caixa De Som Xtreme Mini Bluetooth Usb 40w Primeira Linha,TITLE_ Calça Feminina Cirre Nervurada Nervuras Cintura Alta Justa,...,TITLE_Óculos Vr Box 2.0 Realidade Virtual 3d Android + Controle,TITLE_Óculos Vr Box Realidade Virtual Android Ios+controle #101,TITLE_Óleo 5100 4t 15w50 1 Litro - Motul,TITLE_Óleo Castrol 5w40 Magnatec Stop Start Kit C/ 5,TITLE_Óleo Castrol 5w40 Magnatec Stop Start Kit C/4,"TITLE_Óleo Coco Extra Virgem Ladir 3, Lt + 1 Kg Sal + Vinagre Maçã",TITLE_Óleo Do Motor Incol Alta Performance 25w60 Sl 1l,TITLE_Óleo Do Motor Ipiranga F1 Master 15w40 Sl Semissintético 1l,TITLE_Óleo Pentosin Hc 5w40 Sintético Pentosynth Kit C/5 Promoção,TITLE_Ônibus 1/50 Na Caixa Viagem Internacional Escolar Caravana
361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
#data= df.CATALOG_PRODUCT_ID
#values = np.array(data)
#print(values)
# integer encode
#label_encoder = LabelEncoder()
#integer_encoded = label_encoder.fit_transform(values)
#print(integer_encoded)
# binary encode
#onehot_encoder = OneHotEncoder(sparse=False)
#integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#print(onehot_encoded)


## Actividad 6: 

En caso de tener alguna variable no medida (en nuestro caso `PRICE`) imputar sus valores utilizando kNN.

In [ ]:
from fancyimpute import KNN 
train_cols = list(df)
train = pd.DataFrame(KNN(k=5).complete(train))
train.columns = train_cols

In [160]:
list(df)

['ITEM_ID',
 'SHP_WEIGHT',
 'SHP_LENGTH',
 'SHP_WIDTH',
 'SHP_HEIGHT',
 'ATTRIBUTES',
 'CATALOG_PRODUCT_ID',
 'CONDITION',
 'DOMAIN_ID',
 'PRICE',
 'SELLER_ID',
 'TITLE']

## Actividad 7:

Medir las distribuciones de las variables como histogramas, realizar normalizaciones e identificar outliers con los métodos vistos en clase. Hacer análisis de estos outliers y considerar si sería correcto o no eliminarlos del dataset. Sugerencia: Identificar outliers de las columnas `SHP_WEIGHT` y `SHP_VOLUME`, donde `SHP_VOLUME` se define como el producto de las dimensiones.

In [162]:
# creación de la variable SHP_VOLUME
df["SHP_VOLUME"] = df["SHP_LENGTH"] * df["SHP_WIDTH"] * df["SHP_HEIGHT"]

In [163]:
df.sample(3)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,TITLE,SHP_VOLUME
1550,DF0S84ZAIO,200.0,26.0,16.0,16.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-SUPPLEMENTS,450.0,DZ73J4VU1W,Lida Azul Americano Daidaihua Original 100% Na...,6656.0
9288,S73MMKSRZI,202.0,25.0,20.0,15.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-AEROBICS_AND_FITNESS_EQUIPMENT,119.9,KROHNJ9420,Relogio Smart F3 Pedometro Termometro Android ...,7500.0
7718,VXJ4J184Z3,700.0,20.0,20.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-MOTORCYCLE_REARVIEW_MIRRORS,179.9,LHQEDCF18C,Kit Manete Manopla Retrovisor Lander 250 Ate 0...,4000.0
